In [1]:
from javascript import require, On, Once, AsyncTask, once, off, globalThis
import json
import logging
import time

import building

MINECRAFT_VERSION = '1.20.2'

craft = require('./control_primitives/craftItem.js')
explore = require('./control_primitives/exploreUntil.js')
globalThis.explore = explore
move = require('./control_primitives/goToPosition.js')
kill = require('./control_primitives/killMob.js')
collectPosition = require('./control_primitives/mineBlockPosition.js')
collectType = require('./control_primitives/mineBlockType.js')
pickupDroppedItem = require('./control_primitives/pickupDroppedItem.js')
place = require('./control_primitives/placeItem.js')
shoot = require('./control_primitives/shoot.js')
smelt = require('./control_primitives/smeltItem.js')
chest = require('./control_primitives/useChest.js')

Vec3 = require('vec3').Vec3
mineflayer = require('mineflayer')
pathfinder = require('mineflayer-pathfinder')
pvp = require('mineflayer-pvp').plugin
mcData = require('minecraft-data')(MINECRAFT_VERSION)
globalThis.mcData = mcData
armorManager = require("mineflayer-armor-manager")
autoeat = require('mineflayer-auto-eat').plugin
collectblock = require('mineflayer-collectblock').plugin
hawkeye = require('minecrafthawkeye')
toolPlugin = require('mineflayer-tool').plugin
# mineflayerViewer = require('prismarine-viewer')

RANGE_GOAL = 1

# Load bot
def load_bot(username=None):
  # Create bot
  if not username:
    random_number = 176
    username = f'poo_bucket{random_number}'

  bot = mineflayer.createBot({ 
      'host': 'localhost',
      'port': 22222,
      'username': username, 
      'hideErrors': False 
  })

  once(bot, 'login')
  bot.chat('I spawned')
  #   mineflayerViewer(bot, { 'port': 3000, 'firstPerson': True })

  # Load plugins
  bot.loadPlugin(collectblock)
  bot.loadPlugin(hawkeye)
  bot.loadPlugin(pathfinder.pathfinder)
  bot.loadPlugin(pvp)
  bot.loadPlugin(toolPlugin)

  globalThis.mcData = mcData
  globalThis.pathfinder = pathfinder
  movements = pathfinder.Movements(bot, mcData)
  bot.pathfinder.setMovements(movements)

  # Auto armor
  bot.loadPlugin(armorManager)
  bot.armorManager.equipAll()

  # Auto eat
  bot.loadPlugin(autoeat)
  bot.autoEat.options.priority = 'foodPoints'
  bot.autoEat.options.startAt = 14
  bot.autoEat.options.bannedFood.push('golden_apple')
  
  return bot, movements

In [2]:
bot, movements = load_bot('poo_bucket176')
globalThis.bot = bot

[JSE] Mineflayer detected that you are using a deprecated event (physicTick)! Please use this event (physicsTick) instead.



In [3]:
import memory
import perception
import env_info
# import action

percept = perception.Module()
mem = memory.Module()
# act = action.Module()

c:\Users\g-pc\AppData\Local\Programs\Python\Python310\lib\site-packages\langchain_core\_api\deprecation.py:117: LangChainDeprecationWarning: The class `langchain_community.embeddings.openai.OpenAIEmbeddings` was deprecated in langchain-community 0.1.0 and will be removed in 0.2.0. An updated version of the class exists in the langchain-openai package and should be used instead. To use it run `pip install -U langchain-openai` and import as `from langchain_openai import OpenAIEmbeddings`.
  warn_deprecated(


In [4]:
from javascript import require, globalThis
findAndParseJsonLikeText = require('json-like-parse')

In [5]:
import action_llms

llms = action_llms.get_llms()
bot_functions = json.loads(open('bot_functions.json').read())

previous_action_outcomes = []
current_goal_progress = ''

In [6]:
goal = percept.perceive(bot, 'I have just successfully collected some wood. What should my new Goal be?')
# goal = percept.perceive(bot, 'What is the location of the crafting table near me?')

# goal='COMPLETED: After evaluating the potential goals visible in the surrounding area and considering the resources and challenges present, my new goal in Minecraft will be to focus on resource gathering. Specifically, I will prioritize gathering wood from the nearby trees to craft essential tools, build structures, and create torches for illumination. This goal aligns with the immediate environment and will set a solid foundation for further exploration and survival preparation.'
goal

response="self.askLLM('What should my new goal be after collecting wood?')"
[['Attempt Number: 1', "self.askLLM('What should my new goal be after collecting wood?'): After collecting wood, your next goal could be to build a shelter to protect yourself from mobs and the elements. You could also focus on gathering resources such as stone, coal, and iron to craft better tools and armor. Additionally, you could start exploring the world to find new biomes, villages, and dungeons to discover and conquer."]]
response="self.findBlockType(bot, 'stone', 20)"
[['Attempt Number: 1', "self.askLLM('What should my new goal be after collecting wood?'): After collecting wood, your next goal could be to build a shelter to protect yourself from mobs and the elements. You could also focus on gathering resources such as stone, coal, and iron to craft better tools and armor. Additionally, you could start exploring the world to find new biomes, villages, and dungeons to discover and conquer."], ['Attempt Nu

response='COMPLETED: After collecting wood, the new goal should be to utilize the wood to craft essential tools like a pickaxe, axe, and shovel, set a goal to build a shelter or house using the wood as the primary building material, and identify the need for additional resources such as stone, coal, and iron to craft advanced tools and items.'
Completed task: : After collecting wood, the new goal should be to utilize the wood to craft essential tools like a pickaxe, axe, and shovel, set a goal to build a shelter or house using the wood as the primary building material, and identify the need for additional resources such as stone, coal, and iron to craft advanced tools and items.


'COMPLETED: After collecting wood, the new goal should be to utilize the wood to craft essential tools like a pickaxe, axe, and shovel, set a goal to build a shelter or house using the wood as the primary building material, and identify the need for additional resources such as stone, coal, and iron to craft advanced tools and items.'

In [7]:
while True:
    
    perception_query = llms['generate_perception_query'].invoke({
        'goal': goal,
        'current_environment': env_info.getPromptInfo(bot),
        'previous_action_outcomes': previous_action_outcomes,
        'current_goal_progress': current_goal_progress
    }).content

    print(f'{perception_query=}')

    answer = percept.perceive(bot, perception_query)

    while True:
        try:
            new_action = llms['generate_next_best_action'].invoke({
                'goal': goal,
                'current_environment': env_info.getPromptInfo(bot),
                'previous_action_outcomes': previous_action_outcomes,
                'current_goal_progress': current_goal_progress,
                'question': perception_query,
                'answer': answer,
                'bot_functions': bot_functions
            }).content

            print(f'{new_action=}')


            # Get before
            previous_environment = env_info.getPromptInfo(bot)
            # Execute action
            # Note, change this to return a thing which we can add into the current environment type beat
            # ie change all control primitives to return a string of accomplishments
            exec(new_action)
            break
        except:
            previous_action_outcomes.append([f'Action Number {len(previous_action_outcomes)}', new_action, 'Outcome:', 'Failed to execute action due to improper syntax! Do not make this syntax mistake again.'])

    changes_json = llms['summarize_environment_changes'].invoke({
        'goal': goal,
        'action': new_action,
        'previous_environment': previous_environment,
        'current_environment': env_info.getPromptInfo(bot),
        'previous_goal_progress': current_goal_progress
    }).content

    changes = findAndParseJsonLikeText(changes_json)[0]

    print(f'{changes=}')

    current_goal_progress = changes['new_goal_progress']
    environment_changes = changes['environment_changes']
    previous_action_outcomes.append([f'Action Number {len(previous_action_outcomes)}', new_action, 'Outcome:', environment_changes])

    # Try to gather any new memories
    new_memories = llms['gather_new_memories'].invoke({
        'current_environment': env_info.getPromptInfo(bot),
        'action': new_action,
        'environment_changes': environment_changes,
        'new_goal_progress': current_goal_progress
    }).content

    print(f'{new_memories=}')

    new_memories = findAndParseJsonLikeText(new_memories)[0]
    try:
        collection_name, memory = new_memories['memory1'].split(': ')[0], new_memories['memory1'].split(': ')[1]
        mem.store_memory(memory, collection_name)
        collection_name, memory = new_memories['memory2'].split(': ')[0], new_memories['memory2'].split(': ')[1]
        mem.store_memory(memory, collection_name)
    except:
        pass

perception_query='Perception Module, based on the current environment and my inventory, what is the closest source of stone, coal, or iron that I can gather to craft advanced tools and items to progress towards my goal of building a shelter or house?'
response="self.findBlockType(bot, 'stone', 32)"
[['Attempt Number: 1', "self.findBlockType(bot, 'stone', 32) was found in the world at: Vec3 { \x1bx\x1b: \x1b34\x1b, \x1by\x1b: \x1b64\x1b, \x1bz\x1b: \x1b-32\x1b }!"]]
response="self.findBlockType(bot, 'iron_ore', 32) # Search for nearby iron ore blocks."
[['Attempt Number: 1', "self.findBlockType(bot, 'stone', 32) was found in the world at: Vec3 { \x1bx\x1b: \x1b34\x1b, \x1by\x1b: \x1b64\x1b, \x1bz\x1b: \x1b-32\x1b }!"], ['Attempt Number: 2', "self.findBlockType(bot, 'iron_ore', 32) # Search for nearby iron ore blocks. was found in the world at: Vec3 { \x1bx\x1b: \x1b26\x1b, \x1by\x1b: \x1b65\x1b, \x1bz\x1b: \x1b-32\x1b }!"]]


response='COMPLETED: The closest sources of stone and iron have been located. Stone is at Vec3(34, 64, -32) and iron ore is at Vec3(26, 65, -32). Now, prioritize gathering iron for crafting advanced tools.'
Completed task: : The closest sources of stone and iron have been located. Stone is at Vec3(34, 64, -32) and iron ore is at Vec3(26, 65, -32). Now, prioritize gathering iron for crafting advanced tools.
new_action='collectPosition.mineBlockPosition(bot, Vec3(26, 65, -32), timeout=500000) # will mine the iron ore at position (26, 65, -32)'
